In [1]:
import cv2
import pytesseract
import fastwer
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image, display

In [156]:
# PREPROCESSING IMAGES 

import numpy as np
import cv2

def preprocess_image(img):
    # Rescale the image, if needed.
    img = np.array(img)
    img = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)
    # Convert to gray
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply dilation and erosion to remove some noise
    #kernel = np.ones((1, 1), np.uint8)
    #img = cv2.dilate(img, kernel, iterations=1)
    #img = cv2.erode(img, kernel, iterations=1)
    # Apply blur to smooth out the edges
    #img = cv2.GaussianBlur(img, (5, 5), 0)
    # Apply threshold to get image with only b&w (binarization)
    #img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    return img

In [141]:
# TESSERACT MAIN 

# Tesseract

# Convert document from pdf to image https://stackoverflow.com/questions/29657237/tesseract-ocr-pdf-as-input
import cv2
import pdf2image
import numpy as np
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract


def pdf_to_img(pdf_file, FIRST_PAGE, LAST_PAGE):
    return pdf2image.convert_from_path(pdf_file, first_page= FIRST_PAGE, last_page=LAST_PAGE)

def ocr_core(file):
    pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'
    text = pytesseract.image_to_string(file, lang='spa')
    return text


def print_pages(pdf_file):
    images = pdf_to_img(pdf_file, first_page = 20, last_page = 21)
    for pg, img in enumerate(images):
        img = preprocess_image(img)
        #display(img)
        text = ocr_core(img)
        print('next')


In [142]:
# book_path = 'C:/Users/Hp/OneDrive/Escritorio/libro_1.pdf'
# print_pages(book_path)

In [143]:
# Show the sample images that we will work on (Need to create a folder to upload sample images in Colab environment)
txt_testing = os.listdir('C:/Users/Hp/PycharmProjects/ImageToTextArgentina/data')
txt_testing = sorted(txt_testing)
txt_testing = [doc for doc in txt_testing if '.txt' in doc] 

In [144]:
pdf_list = os.listdir('C:/Users/Hp/PycharmProjects/ImageToTextArgentina/data')
pdf_list = sorted(pdf_list)
pdf_list = [doc for doc in pdf_list if '.pdf' in doc] 

In [170]:
pdf_list 
pdf_list = ['libro_1.pdf','libro_2.pdf', 'libro_3.pdf']

In [179]:
pdf_dict = {
'libro_1.pdf': {'first_page': 20, 'last_page': 21, 'sample': 'libro_1_sample.txt'},
'libro_2.pdf': {'first_page': 1, 'last_page': 2, 'sample': 'libro_2_sample.txt'},
'libro_3.pdf': {'first_page': 20, 'last_page': 21, 'sample': 'libro_3_sample.txt'}
}

In [180]:
# Create empty dataframe to store output
df_output = pd.DataFrame(columns = ['pdf_filename', 'sample_filename', 'txt_reference', 'ocr_output', 'cer', 'wer'])

In [181]:
df_output['pdf_filename'] = pdf_list

In [186]:
for num, pdf in enumerate(pdf_list):
    first_page = pdf_dict[pdf]['first_page']
    last_page = pdf_dict[pdf]['last_page']
    images = pdf_to_img('C:/Users/Hp/PycharmProjects/ImageToTextArgentina/data/' + pdf, first_page, last_page)
    with open('C:/Users/Hp/PycharmProjects/ImageToTextArgentina/data/' + pdf_dict[pdf]['sample'],  'r', encoding='utf-8') as f :
        reference_text = f.read()
    output_text = ''
    for pg, img in enumerate(images):
        #img = preprocess_image(img)
        #display(img)
        output_aux_text = ocr_core(img)
        output_text += output_aux_text
        #print(pdf)
        #print(img)
        #print(output_aux_text)
    output_text = output_text.replace("\n", "")
    output_text = output_text.replace("-\n", "")
    reference_text = reference_text.replace("\n", "")
    reference_text = reference_text.replace("-\n", "")
    cer = fastwer.score_sent(output_text, reference_text, char_level=True)
    wer = fastwer.score_sent(output_text, reference_text, char_level=False)
    df_output.loc[df_output['pdf_filename'] == pdf, 'sample_filename'] = pdf_dict[pdf]['sample']
    df_output.loc[df_output['pdf_filename'] == pdf, 'txt_reference'] = reference_text[0:200]
    df_output.loc[df_output['pdf_filename'] == pdf, 'ocr_output'] = output_text[0:200]
    df_output.loc[df_output['pdf_filename'] == pdf, 'cer'] = cer
    df_output.loc[df_output['pdf_filename'] == pdf, 'wer'] = wer


    
    
    

In [187]:
df_output

,pdf_filename,sample_filename,txt_reference,ocr_output,cer,wer
0,libro_1.pdf,libro_1_sample.txt,do con la importancia de la obra. Al principio...,do con la importancia de la obra. Al principio...,3.1521,22.3529
1,libro_2.pdf,libro_2_sample.txt,"a-Fe20 3, consiste en una formación hexagonal ...","FerritasaA—Fe,O,, consiste en una formación he...",4.3925,19.8805
2,libro_3.pdf,libro_3_sample.txt,"23 lante, cuando la leche se cambia por otra a...","— aeJlante, euando la leche se cambia por otra...",7.6879,34.202


In [ ]:
#for index, row in df_output.iterrows():
#    filename = row['img_filename']
#    ref = row['ref_text']
#    output = row['ocr_output']
#    cer = fastwer.score_sent(output, ref, char_level=True)
#    wer = fastwer.score_sent(output, ref, char_level=False)
#    df_output.loc[df_output['img_filename'] == filename, 'cer'] = round(cer,2) # Round value to 2 decimal places
#    df_output.loc[df_output['img_filename'] == filename, 'wer'] = round(wer,2)

#df_output

In [ ]:
# Overall performances
#mean_cer = df_output['cer'].mean()
#mean_wer = df_output['wer'].mean()
#print(f'Mean CER = {mean_cer}%, Mean WER = {mean_wer}%')